## Aula 03 - Filtragem Baseada em Conteúdo - Exercícios

In [2]:
import pandas as pd
import numpy as np
import re

### Importar base de dados

In [3]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar (1).gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [4]:
movies = pd.read_csv('./dataset/movies_sample.csv').dropna()
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [5]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [6]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.movieId] = row.title


### Divisão da base em treino e teste

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

In [9]:
train

,userId,movieId,rating,title
51276,2985,88,2.5,"Prince & Me, The (2004)"
125583,7302,222,5.0,"Earrings of Madame de..., The (Madame de...) (..."
150073,8729,82,2.0,Australia (2008)
186571,10853,17,3.0,Chasing Amy (1997)
38278,2230,27,3.0,Zero Effect (1998)
...,...,...,...,...
162091,9437,12,4.0,Star Wars: Episode V - The Empire Strikes Back...
175638,10230,16,2.0,Rush Hour 2 (2001)
95816,5556,41,4.5,Dangerous Beauty (1998)
100879,5846,8,2.0,"Karate Kid, The (1984)"


In [10]:
test

,userId,movieId,rating,title
31477,1836,60,2.5,She's the Man (2006)
148621,8646,33,3.5,North by Northwest (1959)
25190,1464,19,4.5,Reservoir Dogs (1992)
91518,5315,33,5.0,North by Northwest (1959)
113446,6571,18,4.0,Life Is Beautiful (La Vita è bella) (1997)
...,...,...,...,...
18611,1083,33,4.0,North by Northwest (1959)
50787,2952,80,4.5,Fog of War: Eleven Lessons from the Life of Ro...
141206,8216,10,4.0,Dark City (1998)
131571,7648,22,4.0,Natural Born Killers (1994)


***Exercício 01:*** Aplique a filtragem baseada em conteúdo (ItemAttributeKNN do CaseRecommender) com as tags associadas aos filmes da base. Utilize Jaccard como métrica de similaridade e k=5 vizinhos para predição.

Documentação do ItemAttributeKNN: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [18]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

In [19]:
movies_tags

,userId,movieId,tag,timestamp_y
0,18,34,Gregory Peck,1329962459
1,18,34,need to own,1329962471
2,18,34,romantic comedy,1329962476
3,18,34,Rome,1329962490
4,18,34,royalty,1329962474
...,...,...,...,...
6269,11075,129,ambitious,1423178287
6270,11075,19,nonlinear,1407271610
6271,11075,19,Quentin Tarantino,1407271608
6272,11075,129,Self-Indulgent,1423178219


In [20]:
metadata_df = movies_tags.groupby("movieId").agg(
        {
            "tag": lambda tags: "|".join(list(tags))
        }).reset_index()

In [21]:
metadata_df[["movieId", "tag"]].to_csv('items_genres.dat', index=False, sep='\t', header=False)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

In [22]:
train

,userId,movieId,rating,title
51276,2985,88,2.5,"Prince & Me, The (2004)"
125583,7302,222,5.0,"Earrings of Madame de..., The (Madame de...) (..."
150073,8729,82,2.0,Australia (2008)
186571,10853,17,3.0,Chasing Amy (1997)
38278,2230,27,3.0,Zero Effect (1998)
...,...,...,...,...
162091,9437,12,4.0,Star Wars: Episode V - The Empire Strikes Back...
175638,10230,16,2.0,Rush Hour 2 (2001)
95816,5556,41,4.5,Dangerous Beauty (1998)
100879,5846,8,2.0,"Karate Kid, The (1984)"


In [18]:
metadata_df

,movieId,tag
0,0,adapted from:book|war|marooned|stranded|alien|...
1,1,gay|London|cheesy|gay romance|adolescent gays|...
2,2,Disney|Paris|jazz|animated|Animated|Disney ani...
3,3,serial killer|Nudity (Topless)|insanity|Christ...
4,4,social commentary|documentary critique|documen...
...,...,...
226,362,Sergio Arau
227,372,abortion|cult film|Thailand|revenge|parody|hor...
228,375,seen 2014|comedy
229,382,Alex De Rakoff


In [16]:
test

,userId,movieId,rating,title
31477,1836,60,2.5,She's the Man (2006)
148621,8646,33,3.5,North by Northwest (1959)
25190,1464,19,4.5,Reservoir Dogs (1992)
91518,5315,33,5.0,North by Northwest (1959)
113446,6571,18,4.0,Life Is Beautiful (La Vita è bella) (1997)
...,...,...,...,...
18611,1083,33,4.0,North by Northwest (1959)
50787,2952,80,4.5,Fog of War: Eleven Lessons from the Life of Ro...
141206,8216,10,4.0,Dark City (1998)
131571,7648,22,4.0,Natural Born Killers (1994)


In [17]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', similarity_metric="jaccard", k_neighbors=5, as_similar_first=True).compute()

NameError: name 'ItemAttributeKNN' is not defined

### Preparação para o exercício 2 - Download e extração de metadados multimídia

In [13]:
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
! tar -xvzf ml-20m-features.tar.gz

100% [......................................................] 5996435 / 5996435
Saved under ml-20m-features.tar (2).gz
features/
features/._m4infus_max_histogram_300_sn.arq
features/m4infus_max_histogram_300_sn.arq
features/._mm_avg_histogram_100_sn.arq
features/mm_avg_histogram_100_sn.arq
features/._visual_histogram_100_sn.arq
features/visual_histogram_100_sn.arq
features/._visual_histogram_50_sn.arq
features/visual_histogram_50_sn.arq
features/._aural_histogram_50.arq
features/aural_histogram_50.arq
features/._mm_max_histogram_300.arq
features/mm_max_histogram_300.arq
features/._m4infus_max_histogram_50.arq
features/m4infus_max_histogram_50.arq
features/._mm_max_histogram_100.arq
features/mm_max_histogram_100.arq
features/._mm_max_histogram_50_sn.arq
features/mm_max_histogram_50_sn.arq
features/._visual_histogram_100.arq
features/visual_histogram_100.arq
features/._visual_histogram_300.arq
features/visual_histogram_300.arq
features/._aural_histogram_100_sn.arq
features/aural_histogra

In [ ]:
import pickle

with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)
print('No. movies: ' + str(len(visualHistograms)))
print('Size of each word: ' + str(len(visualHistograms[0])))

In [ ]:
visualHistograms[0]

***Exercício 02:*** Como visto, o algoritmo ItemAttributeKNN pode ser usado com diferentes tipos de metadados, como gêneros, tags e palavras no geral. Mais do que isso, podemos adaptá-lo também para que a similaridade entre itens seja feita com base em informações multimídia, como imagens, áudio, etc. 

A base de dados utilizada até o momento, ml-20m-compact.tar.gz possui, além das interações de usuários com filmes, uma série de arquivos que contém informações multimídia que foram extraídas dos trailers de cada filme. Esses arquivos estão condensados no zip ml-20m-features.tar.gz, o qual foi feito o download e extraído acima. 

Considere por exemplo o arquivo visual_histogram_50.arq. Ele possui 433 vetores (no. de filmes) de tamanho 50. Podemos pensar que cada vetor desse representa informações visuais (cor, brilho, imagem, etc.) que foram extraídas dos trailers de cada filme. 

Sua tarefa é usar esses vetores de características visuais no cálculo de similaridade entre os filmes, e em seguida, aplicar essas similaridades no algoritmo ItemAttributeKNN para gerar recomendações. 

Dica 1: para calcular a similaridade entre dois vetores pode-se usar o ângulo de cosseno (vide https://en.wikipedia.org/wiki/Cosine_similarity). 

Dica 2: é possível passar para o algoritmo ItemAttributeKNN a matriz de similaridade entre itens, por meio do parâmetro similarity_file=arquivo. Veja em: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [ ]:
import numpy as np

In [ ]:
def cos(a,b):
    numerator = np.dot(a,b)
    denominator = np.linalg.norm(a) * np.linalg.norm(b)
    if (denominator != 0):
        return numerator / denominator 
    else:
        return 0

In [ ]:
def convert_similarity_matrix_to_file(vector_df):
    return pd.DataFrame(vecotr_df, columns=['item1', 'item2', 'sim'])

In [ ]:
def compute_similarity_matrix(vector_df):
    n_vectors = len(vector_df)
    sim_matrix = []
    for i in range(n_vectors):
        for j in range(n_vectors):
            sim = cos(visualHistograms[i], visualHistograms[j])
            row = [i, j, sim]
            sim_matrix.append(row)
    return pd.DataFrame(sim_matrix, columns=['item1', 'item2', 'similarity'])

In [ ]:
def filter_observed_entries(sim_matrix, train_df):
    max_movie_id = max(train_df.movieId.unique())
    min_movie_id = min(train_df.movieId.unique())
    return sim_matrix[
                    (sim_matrix['item1'] <= max_movie_id) & (sim_matrix['item1'] >= min_movie_id)
                    &
                    (sim_matrix['item2'] <= max_movie_id) & (sim_matrix['item2'] >= min_movie_id)
                    ]

In [ ]:
sim_matrix = compute_similarity_matrix(visualHistograms)

In [ ]:
sim_matrix = filter_observed_entries(sim_matrix, train)

In [ ]:
sim_matrix.to_csv('sim_matrix.dat', index=False, sep='\t', header=False)

In [ ]:
ItemAttributeKNN('train.dat', 'test.dat', similarity_file='sim_matrix.dat', k_neighbors=5, as_similar_first=True).compute()

***Exercício 03:*** Implementar uma função que retorna a probabilidade de um item ser relevante para um usuário, considerando os gêneros dos filmes. Utilize métodos probabilísticos. 
- A partir do conjunto de treinamento, obter todas as interações do usuário u.
- Rotular as notas desse usuário como: item relevante se nota >=3 e não relevante se nota < 3. 
- Dado um item do conjunto de teste, aplicar o Teorema de Bayes com suavização de Laplace. Utilizar os gêneros associados. 
- Retornar se o item é ou não relevante, e em seguida, comparar o resultado com a nota real que esse usuário deu para o item (disponível no conjunto de teste).

In [ ]:
prob_train = train.copy()

In [ ]:
prob_train["is_relevant"] = prob_train["rating"] >= 3

In [ ]:
prob_train["movieId"] = prob_train["movieId"].map(map_items)

In [ ]:
movies["movieId"] = movies["movieId"].map(map_items)

In [ ]:
prob_train

In [ ]:
movies_with_genres = pd.merge(prob_train, movies, on=["movieId", "title"])

In [ ]:
# TODO

***Exercício 04:*** No notebook de exemplos, existe uma implementação de Filtragem Baseada em Conteúdo usando Multi-Layer Perceptron como um regressor (MLPRegressor) que prevê a nota de cada usuário para filmes ainda não vistos. O algoritmo retorna uma lista de top K filmes com maiores notas. O treinamento é realizado utilizando as notas que o usuário deu para os filmes e seus respectivos gêneros.
- Usando a classe MLPClassifier (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), implemente uma versão que classifica os filmes não vistos como relevante ou não-relevante.
- No conjunto de dados, realize a binarização das notas, de modo que notas acima de 3 são relevantes e notas abaixo de 3 são não-relevantes.
- Retorne os top k filmes mais relevantes.
- Para um usuário qualquer da base (ou algum outro usuário fictício), analise subjetivamente a qualidade das recomendações, comparando os modelos MLPRegressor e MLPClassifier. 

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def preproc_genres(movies):

    mlb = MultiLabelBinarizer()
    
    movies['genres'] = movies['genres'].replace({"(no genres listed)": ""})
    movies['genres_transformed'] = movies['genres'].apply(lambda genre: genre.split('|'))

    genre_encoded = mlb.fit_transform(movies['genres_transformed'])
    genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

    return pd.concat([movies, genre_df], axis=1).drop(["genres", "genres_transformed"], axis=1)

In [ ]:
def preproc_movies_ids(movies):
    return movies["movieId"].dropna().astype(int).map(map_items).dropna().astype(int)

In [ ]:
movies = preproc_genres(movies)

In [ ]:
from sklearn.neural_network import MLPClassifier

def build_models_dict(train_ratings):
    # Inicializar objetos para armazenar os modelos
    user_models = {}
    # Para cada usuário, treinar um modelo
    train_ratings["is_relevant"] = train_ratings["rating"].apply(lambda rate: rate >= 3)
    for user_id in train_ratings['userId'].unique():
        # Selecionar filmes avaliados
        user_ratings = train_ratings[(train_ratings['userId'] == user_id)]
    
        # Obter os filmes avaliados pelo usuário e seus respectivos gêneros
        user_movies = pd.merge(user_ratings, movies, on=["movieId", "title"])
    
        if len(user_movies) > 1:  # Treinar apenas se houver mais de um filme avaliado
            # Preparar os dados para treinamento da MLP
            X = user_movies.drop(['userId', 'movieId', 'rating', 'is_relevant', 'title'], axis=1)
            y = user_movies['is_relevant']
            # Treinar a MLP
            mlp = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', max_iter=1000, random_state=42)
            mlp.fit(X, y)
    
            # Salvar o modelo e o scaler do usuário
            user_models[user_id] = mlp
    return user_models

In [ ]:
def recommend_movies(user_id, movies_df, ratings, user_models, k=10):
    movies = movies_df.copy()
    if user_id in user_models:
        # Obter o modelo do usuário
        mlp = user_models[user_id]
        
        # Prever as avaliações para todos os filmes
        all_movies = movies.drop(['movieId', 'title'], axis=1)        
        predicted_ratings = mlp.predict(all_movies)

        # Adicionar as previsões ao DataFrame original
        movies['predicted_rating'] = predicted_ratings

        # Remover os filmes que o usuário já avaliou
        user_rated_movies = ratings[ratings['userId'] == user_id]['movieId']
        recommended_movies = movies[~movies['movieId'].isin(user_rated_movies)]

        # Ordenar os filmes pelas previsões e exibir os top k
        recommended_movies = recommended_movies.sort_values(by='predicted_rating', ascending=False)
        return recommended_movies[['title', 'predicted_rating']].head(k)
    else:
        return f"User {user_id} does not have enough data to train a model."


In [148]:
user2model = build_models_dict(train)

In [149]:
user2model

{36594: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 90603: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 108362: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 135425: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 27007: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 41929: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 41160: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 1972: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 125125: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 74175: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 131904: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random_state=42),
 114204: MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=1000, random

In [152]:
recommend_movies(20, movies_df=movies, ratings=test, user_models=user2model)

'User 20 does not have enough data to train a model.'

In [114]:
def get_user_watched_movies(user_id, df):
    watched_movies = df[df["userId"] == user_id].dropna()
    return watched_movies

In [124]:
test["userId"] = test["userId"].map(map_users)

In [125]:
test

,userId,movieId,rating,title
31477,1836,44193,2.5,She's the Man (2006)
148621,8646,908,3.5,North by Northwest (1959)
25190,1464,1089,4.5,Reservoir Dogs (1992)
91518,5315,908,5.0,North by Northwest (1959)
113446,6571,2324,4.0,Life Is Beautiful (La Vita è bella) (1997)
...,...,...,...,...
18611,1083,908,4.0,North by Northwest (1959)
50787,2952,7156,4.5,Fog of War: Eleven Lessons from the Life of Ro...
141206,8216,1748,4.0,Dark City (1998)
131571,7648,288,4.0,Natural Born Killers (1994)


In [130]:
user_id = 20
watched_movies = get_user_watched_movies(user_id, df=test)
print("Filmes assistidos pelo usuário e as respectivas notas:")
print(watched_movies)

top_movies = recommend_movies(user_id, k=10, movies_df=movies, ratings=test, user_models=user2model)
print("Top 10 filmes recomendados:")
print(top_movies)

Filmes assistidos pelo usuário e as respectivas notas:
     userId  movieId  rating                            title
302      20     1569     3.0  My Best Friend's Wedding (1997)
304      20     8798     3.0                Collateral (2004)
Top 10 filmes recomendados:
                                                 title  predicted_rating
190                         Seventh Victim, The (1943)          1.492064
242                              Ninja Assassin (2009)          1.420591
331                                      Loaded (2008)          1.420591
143                                  Collateral (2004)          1.420591
236                            Dark Corner, The (1946)          1.411765
79   Fist of Fury (Chinese Connection, The) (Jing w...          1.396893
162                                   Boogeyman (2005)          1.379315
136                                       Frogs (1972)          1.379315
212                                          21 (2008)          1.359931
2